<a href="https://colab.research.google.com/github/daonly/personal/blob/main/%5BConf%5D_physio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install neurokit2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.4 MB/s eta 0:00:00


In [ ]:
pip install bioread

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 819.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=e749f2ef1f23032a631bb16b93291bd4fa2bea23934873ec5a30509e49f36a38
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [22]:
### preparation
import neurokit2 as nk
import bioread as br
import os
import pandas as pd
import numpy as np

### set path
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/Res/Conformity/data')
datapath = r'/content/drive/MyDrive/Res/Conformity/data'
savepath = r'/content/drive/MyDrive/Res/Conformity/analyses'

Mounted at /content/drive


In [ ]:
### import and set raw data: range 설정 필! 한 파일에 두번 돌리지 말 것.

tasklist = ['HRV', 'HCT', 'BlockI', 'BlockII']

for i in range(11,22):

  txtpath = datapath+f'/sub{i}.txt'

  with open(txtpath, "r") as file:
      lines = file.readlines()
  with open(txtpath, "w") as file:
      file.writelines(lines[23:])

  data = pd.read_csv(txtpath, delimiter='\t', low_memory=False)
  data = data.iloc[1:]; data = data.iloc[:, :-1]

  signal = []
  for j in range(1, len(data)):
    if data['CH25'][j] == '0' and data['CH25'][j+1] == '5':
      signal.append(j+1)

  for j in range(4):
    slicedf = data.iloc[signal[2*j]:signal[2*j+1]]
    savefile = datapath+f'/sub{i}_{tasklist[j]}.csv'
    slicedf.to_csv(savefile, index=True)


In [ ]:
### HRV: range 설정 필!

hrvdf = pd.DataFrame()

for i in range(11,12):

  ## SubID
  SubID = str(i)
  hrvdf.loc[0, 1] = SubID
  hrvdf.columns = ['SubID']

  ## Compute time-domain indices of HRV
  hrvfile = datapath+f'/sub{SubID}_HRV.csv'
  data = pd.read_csv(hrvfile)
  peaks, info = nk.ecg_peaks(data['CH1'], sampling_rate=2000)
  hrvtime = nk.hrv_time(peaks, sampling_rate=2000, show=False)
  hrvdf = pd.concat([hrvdf, hrvtime], axis=1, ignore_index=False)

  ## Compute frequency-domain indices of HRV
  hrvfreq = nk.hrv_frequency(peaks, sampling_rate=2000, show=False)
  hrvdf = pd.concat([hrvdf, hrvfreq], axis=1, ignore_index=False)

  ## Compute nonlinear indices of HRV
  hrvnonlin = nk.hrv_nonlinear(peaks, sampling_rate=2000, show=False)
  hrvdf = pd.concat([hrvdf, hrvnonlin], axis=1, ignore_index=False)

for i in range(12,22):
  hrvdf1 = pd.DataFrame()

  ## SubID
  SubID = str(i)
  hrvdf1.loc[0, 1] = SubID
  hrvdf1.columns = ['SubID']

  ## Compute time-domain indices of HRV
  hrvfile = datapath+f'/sub{SubID}_HRV.csv'
  data = pd.read_csv(hrvfile)
  peaks, info = nk.ecg_peaks(data['CH1'], sampling_rate=2000)
  hrvtime = nk.hrv_time(peaks, sampling_rate=2000, show=False)
  hrvdf1 = pd.concat([hrvdf1, hrvtime], axis=1, ignore_index=False)

  ## Compute frequency-domain indices of HRV
  hrvfreq = nk.hrv_frequency(peaks, sampling_rate=2000, show=False)
  hrvdf1 = pd.concat([hrvdf1, hrvfreq], axis=1, ignore_index=False)

  ## Compute nonlinear indices of HRV
  hrvnonlin = nk.hrv_nonlinear(peaks, sampling_rate=2000, show=False)
  hrvdf1 = pd.concat([hrvdf1, hrvnonlin], axis=1, ignore_index=False)
  hrvdf = pd.concat([hrvdf, hrvdf1], ignore_index=True)

savefile = savepath+f'/HRVDataII.xlsx'
hrvdf.to_excel(savefile, index=False)

hrvdf

,SubID,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,...,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC
0,11,607.083615,19.313147,9.682396,17.009854,8.808427,17.950680,NaN,NaN,12.774366,...,1.403976,7.032843,1.080892,1.515968,1.343173,2.093447,1.643116,1.841703,2.721823,0.762269
1,12,860.791367,48.373133,26.590820,39.293623,18.564376,44.472491,NaN,NaN,37.161922,...,1.417479,7.621987,1.173796,1.268327,1.395120,1.773950,1.668317,1.927292,2.915207,0.855779
2,13,782.704793,34.303846,13.910690,31.920931,13.107628,32.679817,NaN,NaN,24.148490,...,1.477487,7.444357,1.155557,1.437348,1.376913,2.058928,1.644950,1.833342,3.414579,0.847633
3,14,749.387265,42.489401,17.786358,39.081397,12.982078,41.029513,NaN,NaN,31.647683,...,1.507655,7.760690,1.187982,1.509748,1.312941,2.312294,1.761974,1.868530,3.614837,0.817893
4,15,569.535658,37.473873,8.003309,36.405271,8.834716,36.624371,NaN,NaN,32.121267,...,0.821484,7.625648,0.766732,1.312936,1.366045,1.966092,1.294792,1.604989,2.179101,0.589635
5,16,781.314130,45.702692,25.750339,39.029831,25.312254,40.521166,NaN,NaN,32.304119,...,1.556374,7.748874,1.179410,1.230688,1.328089,1.998785,1.790815,1.882263,3.235646,0.807632
6,17,723.149899,27.854156,16.918560,22.638025,6.960088,26.589074,NaN,NaN,14.498975,...,1.412150,7.346149,0.968924,1.373325,1.378861,1.880221,1.585353,1.669834,2.675921,0.702871
7,18,737.907407,34.882168,23.121916,27.595425,18.901298,30.796574,NaN,NaN,19.747922,...,1.339967,7.550768,0.956101,1.394602,1.280851,2.012298,1.655785,1.748677,2.998383,0.697825
8,19,729.630081,57.070849,19.347933,53.891750,16.702952,55.542816,NaN,NaN,45.438999,...,1.780319,8.025472,1.299490,1.252146,1.339354,2.019156,1.841472,1.930869,3.152010,0.763385
9,20,719.815631,32.047265,15.046017,28.691249,7.370847,31.379126,NaN,NaN,17.825262,...,1.539378,7.527679,0.968827,1.609779,1.363784,2.240536,1.738005,1.621040,3.409729,0.682545


In [25]:
### HCT: range 설정 필!

hctdf = pd.DataFrame()
hctdf.loc[11, 1] = '11'  #여기도 수정 필!
hctdf.columns = ['SubID']

for i in range(11,22):

  ## SubID
  SubID = str(i)
  hctdf.loc[i, 'SubID'] = SubID

  ## Extract basic features
  hctfile = datapath+f'/sub{SubID}_HCT.csv'
  data = pd.read_csv(hctfile)

  processed_df, info = nk.bio_process(ecg=data['CH1'], rsp=data['CH3'], sampling_rate=2000)

  ## Set epochs
  hctpath = datapath+f'/sub{i}_HCT.csv'
  data = pd.read_csv(hctpath, low_memory=False)

  startsignal = []; endsignal = []
  for j in range(len(data)-1):
    if data['CH21'][j] == 0 and data['CH21'][j+1] == 5:
      startsignal.append(j+1)
    elif data['CH21'][j] == 5 and data['CH21'][j+1] == 0:
      endsignal.append(j+1)

  ## count R-peaks in each epoch
  hctdf1 = pd.DataFrame()
  for j in range(6):
    slicedf = processed_df.iloc[startsignal[2*j]:endsignal[2*j]]
    rpeaks = slicedf['ECG_R_Peaks'].sum()
    hctdf1.loc[j+1, 1] = rpeaks

  ## calculate indices for interoceptive sensitivity
  rawcardf = pd.read_excel(datapath+f'/HCT/Cardiac-{i}.xls', header=1)
  rawcardf = rawcardf[['HC.RESP','HCCR.Slider1.Value']].iloc[2:8]
  rawcardf.index = range(1,7)
  hctdf1 = pd.concat([hctdf1, rawcardf], axis=1, ignore_index=False)
  hctdf1['stanIAcc'] = 1 - abs(hctdf1['HC.RESP']-hctdf1[1])/hctdf1['HC.RESP']
  hctdf1['alteIAcc'] = 1 - abs(hctdf1['HC.RESP']-hctdf1[1])*2/(hctdf1['HC.RESP']+hctdf1[1])
  hctdf1['alteCheck'] = hctdf1['HC.RESP'] - hctdf1[1]*2
  if (hctdf1['alteCheck'] > 0).any():
    print(f'Need alteIAcc analyses due to sub{i}')
  hctdf.loc[i, 'stanIAcc'] = hctdf1['stanIAcc'].mean()
  hctdf.loc[i, 'alteIAcc'] = hctdf1['alteIAcc'].mean()
  hctdf.loc[i, 'IS'] = hctdf1['HCCR.Slider1.Value'].mean()
  hctdf.loc[i, 'stanIAw'] = hctdf1['stanIAcc'].corr(hctdf1['HCCR.Slider1.Value'])
  hctdf.loc[i, 'alteIAw'] = hctdf1['alteIAcc'].corr(hctdf1['HCCR.Slider1.Value'])

hctdf = hctdf[['SubID', 'stanIAcc', 'alteIAcc', 'IS', 'stanIAw', 'alteIAw']]
savefile = savepath+f'/HCTDataII.xlsx'
hctdf.to_excel(savefile, index=False)

hctdf

,SubID,stanIAcc,alteIAcc,IS,stanIAw,alteIAw
11,11,0.093771,0.406089,5.333333,0.680553,0.588928
12,12,-1.449100,-0.088272,2.000000,NaN,NaN
13,13,0.312804,0.488941,5.500000,-0.477247,-0.476404
14,14,0.502094,0.614388,6.333333,0.740241,0.749392
15,15,-0.091766,0.299691,6.333333,-0.055086,-0.081307
16,16,0.531739,0.621809,5.833333,-0.287373,-0.282724
17,17,-0.409892,0.203136,3.333333,0.768658,0.776792
18,18,-0.446848,0.166681,4.833333,-0.294854,-0.260247
19,19,0.687530,0.745471,6.833333,0.264359,0.254562
20,20,-0.299438,0.213326,6.333333,0.405182,0.419484
